## Setup

In [ ]:
from specific import *

### Get shifted data

In [ ]:
(
    endog_data,
    exog_data,
    master_mask,
    filled_datasets,
    masked_datasets,
    land_mask,
) = get_offset_data()

### Retrieve previous results from the 'model' notebook

In [ ]:
X_train, X_val, y_train, y_val = data_split_cache.load()
rf = get_model()
masked_train_data = get_mm_data(y_train.values, master_mask, "train")
masked_val_data = get_mm_data(y_val.values, master_mask, "val")

### Predict out-of-sample BA

In [ ]:
n_threads = get_ncpus()
rf.n_jobs = n_threads
with parallel_backend("threading", n_jobs=n_threads):
    predicted_ba = get_mm_data(rf.predict(X_val), master_mask, "val")

In [ ]:
cube_plotting(
    np.mean(masked_val_data - predicted_ba, axis=0),
    cmap="brewer_RdYlBu_11",
    cmap_midpoint=0,
    cmap_symmetric=False,
    boundaries=[-0.01, -0.001, -1e-4, 0, 0.001, 0.01, 0.02],
)

In [ ]:
cube_plotting(
    np.mean(masked_val_data - predicted_ba, axis=0) / np.mean(masked_val_data, axis=0),
    cmap="brewer_RdYlBu_11",
    cmap_symmetric=False,
    boundaries=[-5, -4, -3, -2, 0, 1e-2, 1e-1],
    cmap_midpoint=0,
)

In [ ]:
ba_plotting(predicted_ba, masked_val_data, figure_saver)

### Example timeseries

In [ ]:
# valid_indices = list(zip(*np.where(np.sum(~predicted_ba.mask, axis=0) > 5)))
valid_indices = list(
    zip(
        *np.where(
            (np.max(predicted_ba.data, axis=0) > 0.1)
            & (np.sum(~predicted_ba.mask, axis=0) > 4)
        )
    )
)
print("Nr. valid:", len(valid_indices))

for i in np.random.RandomState(0).choice(
    len(valid_indices), min(100, len(valid_indices)), replace=False
):
    plt.figure(figsize=(7, 3))
    plt.plot(masked_val_data[(slice(None), *valid_indices[i])], label="obs", marker="o")
    plt.plot(predicted_ba[(slice(None), *valid_indices[i])], label="pred", marker="x")
    plt.legend(loc="best")